# Setting the fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 2
alphabet = %w(a b c d)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show); nil
a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
nil

["a", "b", "c", "d"]


In [37]:
comm1 = Group.commutator(a[1], b[1])
comm2 = Group.commutator(a[2], b[2])

Del = (comm1*comm2*Word.new('s')).freeze
p 'Del = ' + Del.show

DelPerms = []
perm = Del.dup
(Del.size).times do |n|
  perm = perm.cyclic_permutation
  DelPerms << perm << perm.inverse
end
DelPerms.freeze
#p DelPerms.map(&:show)

comm3 = Group.commutator(a[1], b[1].inverse)
comm4 = Group.commutator(a[2], b[2].inverse)
Rseq = (comm3*comm4).flatten!.factors.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:3: warning: already initialized constant Del
<main>:3: warning: previous definition of Del was here


"Del = ((abAB)(cdCD))(s)"


<main>:6: warning: already initialized constant DelPerms
<main>:6: warning: previous definition of DelPerms was here
<main>:17: warning: already initialized constant Rseq
<main>:17: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d"]


In [4]:
w = Word.new('abDdcdbABcd'); nil

# The Algorithm

### (i) Contracting the word

In [5]:
# (i) contract
reduced = w.contract.dup
reduced.show

"abcdbABcd"

### (ii) removing subwords

In [6]:
def reduce_once(myw)
  facs = myw.flatten.factors
  (myw.size+1).times do |i|
    w1 = Word.new(facs[0,i])
    w2 = Word.new(facs.drop(i))
    original_size = myw.size
    for perm in DelPerms
      wc = (w1*perm*w2).deep_copy.contract
      if (wc.size < original_size) then
        myw = wc 
        break
      end
    end
    if (myw.size < original_size) then
      return myw
      break
    end
  end
  return nil    
end
    
def reduce(myword)
  while not myword.nil?
    prev = myword
    myword = reduce_once(myword)
  end
  return prev.contract
end
nil

In [7]:
p 'w = ' + w.show + ' and w.size = ' + w.size.to_s
p reduce(w).show

"w = abcdbABcd and w.size = 9"
"abcdASdc"


"abcdASdc"

### (iii) removing subwords part2

In [8]:
# not necessary?

## The algorithm

In [9]:
gens_ex = Gens + [Letter.new('s')]
for g in gens_ex[1..-1]
  p g.show + ': ' + w.count(g).to_s
  p g.inverse.show + ': ' + w.count(g.inverse).to_s
end;nil

"a: 1"
"A: 1"
"b: 2"
"B: 1"
"c: 2"
"C: 0"
"d: 2"
"D: 0"
"s: 0"
"S: 0"


In [10]:
def P(w)
  count = 0
  facs = w.flatten.factors
  facs.each_with_index do |f, i|
    succ = (i < facs.size - 1) ? facs[i+1] : facs[0]
    pair = [f.inverse, succ]
    # p pair.map(&:show)
    idxs = pair.map{|f| Rseq.find_index(f)}
    # p idxs
    count += 1 if idxs[0] < idxs[1]
  end
  return count
end

:P

In [11]:
#p w.show, Rseq.map(&:show)
P(w)

7

In [45]:
#myw = Word.new(a[1], b[1].inverse).contract
myw = w
subs = myw.split(1)
ww = subs[0]*(subs[1].inverse)
p "w = #{myw.show}, div = #{subs.map(&:show)}, ww = #{ww.show}"

ww.flatten!
p "P=#{P(ww)}, A_bar=#{a.map{|g| ww.count(g.inverse)}.inject(0, &:+)}, \
B=#{b.map{|g| ww.count(g)}.inject(0, &:+)}, \
S_bar=#{ww.count(Letter.new('s').inverse)}"
nil

"w = abcdbABcd, div = [\"a\", \"bcdbABcd\"], ww = (a)(DCbaBDCB)"
"P=2, A_bar=2, B=1, S_bar=0"
